# Урок 7. Сверточные нейронные сети для анализа текста

## Задание
Берем отызывы за лето (из архива с материалами или предыдущего занятия)

1. Обучить conv сеть для классификации. Рассмотреть 2-а варианта сеточек<br>
    1.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/<br>
    1.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)<br>

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

## Решение.

### Подготовка данных.

In [1]:
import os
import pandas as pd

In [2]:
# !pip install xlrd
# !pip install stop_words

In [3]:
df = pd.read_excel('отзывы за лето.xls')

In [4]:
df = df[['Rating','Content']]
df.head()

,Rating,Content
0,5,It just works!
1,4,В целом удобноное приложение...из минусов хотя...
2,5,Отлично все
3,5,Стал зависать на 1% работы антивируса. Дальше ...
4,5,"Очень удобно, работает быстро."


In [5]:
df['Rating'].value_counts()

5    14586
1     2276
4     2138
3      911
2      748
Name: Rating, dtype: int64

Установи порог отзывов 3 бала. если значение выше порога отзыв положительный. Если ниже- отрицательный.

In [6]:
df['Rating'] = df['Rating'].apply(lambda x: 1 if x > 3 else 0)

In [7]:
df['Rating'].value_counts()

1    16724
0     3935
Name: Rating, dtype: int64

### Предобработка текста.

In [8]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
from sklearn.model_selection import train_test_split
from utils import apostrophe_dict, emoticon_dict, short_word_dict  # см. файл utils.py


In [9]:
max_words = 1000
max_len = 100
num_classes = 1

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

In [10]:
# Теперь повторим это для всех записей.
def replase_words(text,dict_): 
    output = ''
    for word in text.split(' '): # ТОDO не будем делить текст на части будем искать подстроку в строке. это касается только смайликов.
        word = word.strip()
        if word in dict_.keys(): 
            output += ' ' + dict_[word]
        else:
            output += ' ' + word
    return output

In [11]:
df['Content_с'] = df['Content']

In [12]:
df

,Rating,Content,Content_с
0,1,It just works!,It just works!
1,1,В целом удобноное приложение...из минусов хотя...,В целом удобноное приложение...из минусов хотя...
2,1,Отлично все,Отлично все
3,1,Стал зависать на 1% работы антивируса. Дальше ...,Стал зависать на 1% работы антивируса. Дальше ...
4,1,"Очень удобно, работает быстро.","Очень удобно, работает быстро."
...,...,...,...
20654,0,"Ну и шляпа,с роот правами бесполезная прога,ра...","Ну и шляпа,с роот правами бесполезная прога,ра..."
20655,1,Ок,Ок
20656,1,Доволен,Доволен
20657,0,"Песопаснасть, рут ни нужын","Песопаснасть, рут ни нужын"


In [13]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    # Заменим эмодзи на соответствующие им слова.
    txt = re.sub("[\,]","",txt)
    txt = replase_words(txt, emoticon_dict)
     # Заменим сокращения на их полные формы
    txt = replase_words(txt, apostrophe_dict)
    txt = replase_words(txt, short_word_dict)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split()] # if word not in sw]
    # [w for w in text.split() if len(w)>1 and w != "i"])
    return " ".join([w for w in txt if len(w)>1])

df['Content'] = df['Content'].apply(preprocess_text)

**После очитски текста, длина части комментариев оказалась равной нулю!**

In [14]:
df['Text_len'] = [len(text) for text in df['Content']]

In [15]:
df.loc[((df['Text_len']==0) & (df['Rating']==0))]

,Rating,Content,Content_с,Text_len


In [16]:
df.loc[((df['Text_len']==0) & (df['Rating']==1))]

,Rating,Content,Content_с,Text_len
3080,1,,.👍,0
4083,1,,+,0
7479,1,,👍!!!,0
8724,1,,О.,0
15887,1,,N 1,0
16207,1,,во,0


In [17]:
df.loc[(df['Text_len']==0),'Rating'].value_counts() # как быть вот с такими комметариями!

1    6
Name: Rating, dtype: int64

Раделим DataFrame на ренировочную и тестовую выборки !!! 

In [18]:
df_train, df_test = train_test_split(df,test_size=0.3)

In [19]:
train_corpus = " ".join(df_train["Content"])
train_corpus = train_corpus.lower()

In [20]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /home/sergey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [21]:
tokens_filtered = [word for word in tokens if word.isalnum()]

Отфильтруем данные и соберём в корпус N наиболее частых токенов

In [22]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [23]:
tokens_filtered_top[:50]

['приложение',
 'весь',
 'очень',
 'удобно',
 'на',
 'работать',
 'удобный',
 'что',
 'спасибо',
 'отлично',
 'хороший',
 'нравиться',
 'это',
 'отличный',
 'раз',
 'по',
 'телефон',
 'но',
 'после',
 'хорошо',
 'быть',
 'мочь',
 'супер',
 'при',
 'быстро',
 'за',
 'обновление',
 'как',
 'только',
 'так',
 'просто',
 'антивирус',
 'пользоваться',
 'не',
 'пароль',
 'сбербанк',
 'вы',
 'устраивать',
 'вход',
 'то',
 'карта',
 'прошивка',
 'пока',
 'свой',
 'из',
 'для',
 'нормальный',
 'банк',
 'есть',
 'рута']

In [24]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [25]:
import numpy as np

def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [26]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["Content"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["Content"]], dtype=np.int32)
# x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["Content"]], dtype=np.int32)

## Обучим нейронную сеть.

In [27]:
# !pip install keras

In [28]:
import numpy as np
import tensorflow.keras

In [29]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D

In [30]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import TensorBoard 

from tensorflow.keras.callbacks import EarlyStopping  
from sklearn.metrics import accuracy_score 

In [31]:
num_classes = 2
y_train = tensorflow.keras.utils.to_categorical(df_train["Rating"], num_classes)
y_test = tensorflow.keras.utils.to_categorical(df_test["Rating"], num_classes)

In [32]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [39]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [40]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Train on 13014 samples, validate on 1447 samples
Epoch 1/20
13014/13014 [==============================] - 9s 726us/sample - loss: 0.1671 - acc: 0.9351 - val_loss: 0.2258 - val_acc: 0.9129
Epoch 2/20
13014/13014 [==============================] - 8s 636us/sample - loss: 0.1523 - acc: 0.9431 - val_loss: 0.2348 - val_acc: 0.9122


In [41]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)

6198/6198 [==============================] - 1s 96us/sample


In [42]:
y_pred = np.argmax(results,axis = 1)

In [43]:
accuracy_score(df_test['Rating'].to_numpy(), y_pred)

0.9080348499515973

пример классификации текста с помощю библиотеки взят отсюда -- https://github.com/ltgoslo/simple_elmo/blob/master/simple_elmo/examples/text_classification.py